In [ ]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
import pandas as pd
import tenseal as ts
import tensorflow.keras.backend as K

In [ ]:
def build_mlp(input_dim, num_layers, neurons_list, learning_rate):


    def poly_sigmoid(x):
        return 0.5 + 0.25*x - 0.0208*(x**3)

    model = Sequential()
    model.add(tf.keras.Input(shape=(input_dim,)))

    # First layer
    model.add(Dense(neurons_list[0]))
    model.add(tf.keras.layers.Lambda(poly_sigmoid))

    # Hidden layers
    for i in range(1, num_layers):
        model.add(Dense(neurons_list[i]))
        model.add(tf.keras.layers.Lambda(poly_sigmoid))

    # Output layer
    model.add(Dense(1))
    model.add(tf.keras.layers.Lambda(poly_sigmoid))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def fitness_fn(params, X_train, y_train, X_val, y_val, class_weight_dict):

    bounds = [
        (1, 2),           # number of layers
        (8, 128),         # neurons in layer 1
        (8, 128),         # neurons in layer 2
        (-4, -1),         # log10 learning rate
        (20, 200)         # epochs
    ]


    params = np.array([np.clip(p, bounds[i][0], bounds[i][1]) for i, p in enumerate(params)])

    # Check for NaN or inf
    if np.any(np.isnan(params)) or np.any(np.isinf(params)):
        print(f"Invalid parameters detected: {params}")
        return float('inf')


    batch_size = 75


    num_layers = int(params[0])
    neurons = [int(params[i]) for i in range(1, 1 + num_layers)]
    learning_rate = 10 ** params[3]
    epochs = int(params[4])


    epochs = max(1, min(epochs, 200))


    print(f"  Testing config: layers={num_layers}, neurons={neurons}, lr={learning_rate:.6f}, batch={batch_size}, epochs={epochs}")

    try:
        model = build_mlp(X_train.shape[1], num_layers, neurons, learning_rate)
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                           class_weight=class_weight_dict, verbose=0, validation_data=(X_val, y_val))


        y_pred = model.predict(X_val, verbose=0)
        y_pred_binary = (y_pred > 0.4).astype(int).flatten()


        sse = np.sum((y_val - y_pred.flatten()) ** 2)

        lambda_reg = 0.001  # Reduced from 0.01


        l1_penalty = 0
        for layer in model.layers:
            if hasattr(layer, 'kernel'):
                l1_penalty += np.sum(np.abs(layer.kernel.numpy()))

        sse_lasso = sse + lambda_reg * l1_penalty


        print(f"    SSE: {sse:.4f}, L1_penalty: {l1_penalty:.4f}, SSE_lasso: {sse_lasso:.4f}")


        del model
        tf.keras.backend.clear_session()

        return sse_lasso

    except Exception as e:
        print(f"Error in fitness function: {e}")
        return float('inf')


In [ ]:
'''def m_mrfo_optimize(X_train, y_train, X_val, y_val, class_weight_dict, population_size=20, max_iter=30):

    import math

    dim = 5
    bounds = [
        (1, 2),          #num of layers
        (8, 128),        # neurons 1
        (8, 128),        # neurons 2
        (-4, -1),        # log10 learning rate
        (20, 200)        # epochs
    ]

    # Initialize popn
    population = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], (population_size, dim))
    fitness = [fitness_fn(ind, X_train, y_train, X_val, y_val, class_weight_dict) for ind in population]
    best_idx = np.argmin(fitness)
    best_pos = population[best_idx].copy()

    beta = 0.005  # spiral coefficient
    alpha = 2     # somersault factor

    print(f"Initial best fitness: {fitness[best_idx]:.4f}")

    for t in range(max_iter):
        print(f"\nIteration {t+1}/{max_iter}")
        a = math.exp(-t / max_iter)  # adaptive coefficient

        for i in range(population_size):
            print(f"  Evaluating individual {i+1}/{population_size}:")

            r1, r2 = np.random.rand(), np.random.rand()
            new_pos = population[i].copy()

            if r1 < 0.5:  # Chain foraging
                leader = best_pos if r2 < 0.5 else population[(i - 1) % population_size]
                new_pos += r1 * (leader - population[i])
            else:  # Cyclone foraging
                if r2 < 0.5:
                    new_pos += beta * np.random.rand(dim) * (best_pos - population[i])
                else:
                    theta = 2 * np.pi * np.random.rand()
                    spiral = beta * np.exp(a * theta) * np.cos(theta)
                    new_pos += spiral * (best_pos - population[i])

            # Somersault foraging
            somersault = population[i] + alpha * (np.random.rand(dim) * best_pos - np.random.rand(dim) * population[i])
            new_pos = (new_pos + somersault) / 2.0

            #new position
            for j in range(dim):
                new_pos[j] = np.clip(new_pos[j], bounds[j][0], bounds[j][1])

            score = fitness_fn(new_pos, X_train, y_train, X_val, y_val, class_weight_dict)

            if score < fitness[i]:
                population[i] = new_pos
                fitness[i] = score
                print(f"  --> IMPROVED: {score:.4f}")

                if score < fitness[best_idx]:
                    best_idx = i
                    best_pos = new_pos.copy()
                    print(f"  --> NEW GLOBAL BEST: {score:.4f}")
            else:
                print(f"  --> No improvement: {score:.4f} vs {fitness[i]:.4f}")

        print(f"Iteration {t+1}/{max_iter}, Best SSE_Lasso: {fitness[best_idx]:.4f}")

    return best_pos, fitness[best_idx]'''

In [ ]:
'''def AOA_optimize(X_train, y_train, X_val, y_val, class_weight_dict, population_size=20, max_iter=30):

    dim = 5

    bounds = [
        (1, 2),           # num of layers
        (8, 128),         # neurons 1
        (8, 128),         # neurons 2
        (-4, -1),         # log10 learning rate
        (20, 200)         # epochs
    ]

    # neurons and epochs should be int
    discrete_params = [0, 1, 2, 4]


    alpha = 5      # Sensitivity parameter
    mu = 0.499     # Control parameter

    # Initialize popn
    population = np.zeros((population_size, dim))
    for i in range(population_size):
        for j in range(dim):
            lb, ub = bounds[j]
            population[i][j] = np.random.uniform(lb, ub)
            # Round discrete parameters
            if j in discrete_params:
                population[i][j] = round(population[i][j])

    # Evaluate initial popn
    fitness_values = []
    print("Evaluating initial population...")
    for i in range(population_size):
        print(f"Individual {i+1}/{population_size}:")
        fitness = fitness_fn(population[i], X_train, y_train, X_val, y_val, class_weight_dict)
        fitness_values.append(fitness)
        print(f"  Final fitness = {fitness:.4f}")

    # Find initial best soln
    best_idx = np.argmin(fitness_values)
    best_fitness = fitness_values[best_idx]
    best_position = population[best_idx].copy()

    print(f"Initial best fitness: {best_fitness:.4f}")

    # Main optimization
    for iteration in range(max_iter):
        print(f"\nIteration {iteration+1}/{max_iter}")

        # Update MOA
        MOA = 1.0 - (iteration / max_iter)

        # Update MOP
        MOP = 1.0 - ((iteration + 1) / max_iter) ** (1.0 / alpha)

        for i in range(population_size):
            print(f"  Evaluating individual {i+1}/{population_size}:")

            # Create new soln
            new_solution = population[i].copy()

            for j in range(dim):
                lb, ub = bounds[j]


                r1 = np.random.rand()
                r2 = np.random.rand()
                r3 = np.random.rand()

                # Exploration phase
                if r1 > MOA:
                    if r2 > 0.5:
                        # Addition operator (A)
                        new_solution[j] = best_position[j] + MOP * ((ub - lb) * mu + lb) * r3
                    else:
                        # Subtraction operator (S)
                        new_solution[j] = best_position[j] - MOP * ((ub - lb) * mu + lb) * r3

                # Exploitation phase
                else:
                    if r3 > 0.5:
                        # Multiplication operator (M)
                        new_solution[j] = best_position[j] * MOP * ((ub - lb) * mu + lb)
                    else:
                        # Division operator (D)
                        divisor = MOP * ((ub - lb) * mu + lb)
                        if abs(divisor) > 1e-10:  # Avoid division by zero
                            new_solution[j] = best_position[j] / divisor
                        else:
                            new_solution[j] = best_position[j]

                # Apply bounds
                new_solution[j] = np.clip(new_solution[j], lb, ub)

                # Handle discrete params
                if j in discrete_params:
                    new_solution[j] = round(new_solution[j])

            # Evaluate new soln
            new_fitness = fitness_fn(new_solution, X_train, y_train, X_val, y_val, class_weight_dict)

            # greedy selection
            if new_fitness < fitness_values[i]:
                population[i] = new_solution
                fitness_values[i] = new_fitness
                print(f"  --> IMPROVED: {new_fitness:.4f}")

                # Update global best
                if new_fitness < best_fitness:
                    best_fitness = new_fitness
                    best_position = new_solution.copy()
                    best_idx = i
                    print(f"NEW GLOBAL BEST: {best_fitness:.4f}")
            else:
                print(f"No improvement: {new_fitness:.4f} vs {fitness_values[i]:.4f}")

        print(f"Iteration {iteration+1}/{max_iter}, Best Fitness: {best_fitness:.4f}")

    return best_position, best_fitness'''

In [ ]:
def SCSO_optimize(X_train, y_train, X_val, y_val, class_weight_dict, population_size=20, max_iter=30):

    dim = 5

    bounds = [
        (1 ,2),          #num of layers
        (8, 128),        # neurons 1
        (8, 128),        # neurons 2
        (-4, -1),        # log10 learning rate
        (20, 200)        # epochs
    ]

    # Initialize popn
    population = np.zeros((population_size, dim))
    for i in range(population_size):
        for j in range(dim):
            population[i, j] = np.random.uniform(bounds[j][0], bounds[j][1])

    # Evaluate initial popn
    fitness_values = []
    print("Evaluating initial population...")
    for i in range(population_size):
        print(f"Individual {i+1}/{population_size}:")
        fitness = fitness_fn(population[i], X_train, y_train, X_val, y_val, class_weight_dict)
        fitness_values.append(fitness)
        print(f"  Final fitness = {fitness:.4f}")

    # Find best soln
    best_idx = np.argmin(fitness_values)
    best_fitness = fitness_values[best_idx]
    best_position = population[best_idx].copy()

    print(f"Initial best fitness: {best_fitness:.4f}")

    # Main optimization
    for iteration in range(max_iter):
        print(f"\nIteration {iteration+1}/{max_iter}")

        for i in range(population_size):
            # Current position
            current_pos = population[i].copy()


            r = np.random.rand()
            R = np.random.rand()

            # Update pos
            if R <= 0.5:
                # Exploitation phase
                gamma = 2 * np.random.rand() - 1  # Random coefficient [-1, 1]
                new_pos = best_position + gamma * np.random.rand(dim) * (best_position - current_pos)
            else:
                # Exploration phase
                if np.random.rand() < 0.5:
                    # Random exploration around best soln
                    new_pos = best_position + np.random.randn(dim) * 0.1 * (bounds[0][1] - bounds[0][0])
                else:
                    new_pos = np.array([np.random.uniform(bounds[j][0], bounds[j][1]) for j in range(dim)])

            #handle NaN/inf
            for d in range(dim):
                if np.isnan(new_pos[d]) or np.isinf(new_pos[d]):
                    new_pos[d] = np.random.uniform(bounds[d][0], bounds[d][1])
                new_pos[d] = np.clip(new_pos[d], bounds[d][0], bounds[d][1])

            # Evaluate new posn
            print(f"  Evaluating individual {i+1}/{population_size}:")
            new_fitness = fitness_fn(new_pos, X_train, y_train, X_val, y_val, class_weight_dict)

            # Update if better
            if new_fitness < fitness_values[i]:
                population[i] = new_pos
                fitness_values[i] = new_fitness
                print(f"  --> IMPROVED: {new_fitness:.4f}")


                if new_fitness < best_fitness:
                    best_fitness = new_fitness
                    best_position = new_pos.copy()
                    best_idx = i
                    print(f"  --> NEW GLOBAL BEST: {best_fitness:.4f}")
            else:
                print(f"  --> No improvement: {new_fitness:.4f} vs {fitness_values[i]:.4f}")

        print(f"Iteration {iteration+1}/{max_iter}, Best Fitness: {best_fitness:.4f}")

    return best_position, best_fitness

In [ ]:
def setup_ckks_context():
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=16384,
        coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 60]
    )
    context.global_scale = 2**40
    context.generate_galois_keys()
    return context

def ckks_encrypt_vector(vec, context):
    return ts.ckks_vector(context, vec.tolist())

def robust_encrypted_sigmoid(enc_x):
    try:
        #simple polynomial approximation: 0.5 + 0.25*x - 0.02*x^2
        x_squared = enc_x * enc_x
        linear_term = enc_x * 0.25
        quadratic_term = x_squared * 0.02
        result = linear_term - quadratic_term + 0.5
        return result

    except Exception as e:
        print(f"Error in robust_encrypted_sigmoid: {e}")
        return enc_x * 0.25 + 0.5

def encrypted_feedforward_with_scale_management(enc_input, weights, biases):
    x = enc_input

    for layer_idx, (weight_matrix, bias_vector) in enumerate(zip(weights, biases)):
        print(f"Processing layer {layer_idx + 1}/{len(weights)}")
        layer_outputs = []

        for j in range(weight_matrix.shape[1]):
            try:
                neuron_sum = None

                if layer_idx == 0:
                    #input is single encrypted vecto
                    weight_column = weight_matrix[:, j]


                    weight_max = np.max(np.abs(weight_column))
                    if weight_max > 2.0:
                        weight_column = weight_column / weight_max
                        scale_factor = weight_max
                    else:
                        scale_factor = 1.0

                    for k, weight_val in enumerate(weight_column):
                        if abs(weight_val) > 1e-4:
                            term = enc_input * float(weight_val)
                            if neuron_sum is None:
                                neuron_sum = term
                            else:
                                neuron_sum = neuron_sum + term


                    if neuron_sum is not None and scale_factor > 1.0:
                        neuron_sum = neuron_sum * scale_factor

                else:
                    #Next layers
                    weight_column = weight_matrix[:, j]
                    weight_max = np.max(np.abs(weight_column))
                    if weight_max > 2.0:
                        weight_column = weight_column / weight_max
                        scale_factor = weight_max
                    else:
                        scale_factor = 1.0

                    for k, weight_val in enumerate(weight_column):
                        if abs(weight_val) > 1e-4:
                            term = x[k] * float(weight_val)
                            if neuron_sum is None:
                                neuron_sum = term
                            else:
                                neuron_sum = neuron_sum + term

                    if neuron_sum is not None and scale_factor > 1.0:
                        neuron_sum = neuron_sum * scale_factor


                if neuron_sum is None:
                    neuron_sum = (enc_input if layer_idx == 0 else x[0]) * 0.0


                bias_val = float(bias_vector[j])
                bias_val = np.clip(bias_val, -2.0, 2.0)
                neuron_output = neuron_sum + bias_val

                # Apply sigmoid
                neuron_output = robust_encrypted_sigmoid(neuron_output)

                layer_outputs.append(neuron_output)

            except Exception as e:
                print(f"Error processing neuron {j} in layer {layer_idx}: {e}")
                default_val = (enc_input if layer_idx == 0 else x[0]) * 0.0 + 0.5
                layer_outputs.append(default_val)

        x = layer_outputs

    return x[0] if len(x) == 1 else x

def preprocess_for_encryption(X_data):
    X_normalized = X_data.copy()
    for i in range(X_normalized.shape[1]):
        col = X_normalized[:, i]
        col_max = np.max(np.abs(col))
        if col_max > 0:
            X_normalized[:, i] = col / col_max

    X_normalized = np.clip(X_normalized, -1.0, 1.0)
    return X_normalized

def extract_dense_weights(model):
    weights = []
    biases = []

    for layer in model.layers:
        if isinstance(layer, Dense):
            w, b = layer.get_weights()
            weights.append(w)
            biases.append(b)

    return weights, biases

In [ ]:
def evaluate_best_model_improved(model, X_test_encrypted, y_test):
    print("Extracting weights and biases from trained model,")
    weights, biases = extract_dense_weights(model)


    normalized_weights = []
    for w in weights:
        w_max = np.max(np.abs(w))
        if w_max > 1.0:
            normalized_weights.append(w / w_max)
        else:
            normalized_weights.append(w)

    # Normalize
    normalized_biases = []
    for b in biases:
        normalized_biases.append(np.clip(b, -5.0, 5.0))

    print(f"Model architecture: {[w.shape for w in normalized_weights]}")
    print(f"Evaluating on {len(X_test_encrypted)} encrypted samples...")

    y_pred_enc = []

    for i, enc_vec in enumerate(X_test_encrypted):
        print(f"Processing sample {i+1}/{len(X_test_encrypted)}")

        try:
            # Forward pass through encrypted network
            pred_enc = encrypted_feedforward_with_scale_management(enc_vec, normalized_weights, normalized_biases)

            # Decrypt prediction
            decrypted_pred = pred_enc.decrypt()

            # Handle both single value and vector output types
            if isinstance(decrypted_pred, list) and len(decrypted_pred) > 0:
                pred_value = decrypted_pred[0]
            else:
                pred_value = float(decrypted_pred)

            y_pred_enc.append(pred_value)

        except Exception as e:
            print(f"Error processing sample {i}: {e}")
            y_pred_enc.append(0.5)  #Default

    # Convert to binary
    y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred_enc]


    acc = accuracy_score(y_test, y_pred_binary)
    balanced_acc = balanced_accuracy_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary, average='binary')
    recall = recall_score(y_test, y_pred_binary, average='binary')
    f1 = f1_score(y_test, y_pred_binary, average='binary')
    sse = mean_squared_error(y_test, y_pred_enc) * len(y_test)

    print(f"Encrypted Accuracy: {acc:.4f}")
    print(f"Encrypted Balanced Accuracy: {balanced_acc:.4f}")
    print(f"Encrypted Precision: {precision:.4f}")
    print(f"Encrypted Recall: {recall:.4f}")
    print(f"Encrypted F1 Score: {f1:.4f}")
    print(f"Encrypted SSE: {sse:.4f}")

    return {
        'accuracy': acc,
        'balanced_accuracy': balanced_acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'sse': sse
    }

In [ ]:
def evaluate_best_model(X_train, y_train, X_test, y_test, best_params, class_weight_dict):
    print("Evaluating model based on optimised config:")


    batch_size = 75

    num_layers = int(best_params[0])
    neurons = [int(best_params[i]) for i in range(1, 1 + num_layers)]
    learning_rate = 10 ** best_params[3]
    epochs = int(best_params[4])

    epochs = max(1, min(epochs, 200))

    print(f"Configuration:")
    print(f"  Number of layers: {num_layers}")
    print(f"  Neurons per layer: {neurons}")
    print(f"  Learning rate: {learning_rate:.6f}")
    print(f"  Batch size: {batch_size} (fixed)")
    print(f"  Epochs: {epochs}")

    model = build_mlp(X_train.shape[1], num_layers, neurons, learning_rate)

    print(f"\nTraining final model...")
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        class_weight=class_weight_dict,
                        verbose=1)

    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()


    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')

    print("Final Model Performance Metrics")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")

    # Calculate final SSE_lasso for comparison
    sse = np.sum((y_test - y_pred_proba.flatten()) ** 2)
    lambda_reg = 0.001
    l1_penalty = 0
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            l1_penalty += np.sum(np.abs(layer.kernel.numpy()))
    sse_lasso = sse + lambda_reg * l1_penalty

    print(f"\nFinal SSE_lasso: {sse_lasso:.4f}")

    return {
        'balanced_acc': balanced_acc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'sse_lasso': sse_lasso,
        'model': model,
        'history': history.history
    }

In [ ]:
def main():
    # Load dataset
    df = pd.read_csv('/content/Post_LDA_dataset.csv')
    X = df.drop(columns=["target"])
    y = df['target'].values
    X_lda = X.values

    print(f"Dataset shape: {X_lda.shape}")
    print(f"Target distribution: {np.bincount(y)}")

    # Split data: train/temp (70/30), then temp -> val/test (15/15)
    X_train, X_temp, y_train, y_temp = train_test_split(X_lda, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    print(f"\nOriginal training set distribution: {np.bincount(y_train)}")

    from sklearn.utils import class_weight

    class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train)

    class_weight_dict = dict(enumerate(class_weights))

    #Optimization call
    best_params, best_fitness = SCSO_optimize(X_train, y_train, X_val, y_val,
                                           class_weight_dict, population_size=20, max_iter=30)

    print("\nOptimization results:")
    print(f"Best Parameters Found: {best_params}")
    print(f"Best Fitness (SSE_lasso): {best_fitness:.4f}")

    # Decode params
    num_layers = int(best_params[0])
    neurons = [int(best_params[i]) for i in range(1, 1 + num_layers)]
    learning_rate = 10 ** best_params[3]
    epochs = int(best_params[4])
    batch_size = 75  # Fixed

    print(f"\nDecoded Parameters:")
    print(f"  Number of layers: {num_layers}")
    print(f"  Neurons per layer: {neurons}")
    print(f"  Learning rate: {learning_rate:.6f}")
    print(f"  Batch size: {batch_size} (fixed)")
    print(f"  Epochs: {epochs}")


    results = evaluate_best_model(X_train, y_train, X_test, y_test,
                                 best_params, class_weight_dict)

    #CKKS encryption
    print("\nSetting up CKKS context...")
    context = setup_ckks_context()

    # Encrypt test data
    print("Encrypting test data...")
    test_subset_size = min(5, len(X_test))
    X_test_subset = X_test[:test_subset_size]
    y_test_subset = y_test[:test_subset_size]

    X_test_normalized = preprocess_for_encryption(X_test_subset)
    X_test_encrypted = [ckks_encrypt_vector(row, context) for row in X_test_normalized]

    print(f"Building and training final model for encryption evaluation.")
    model = results['model']  l

    print("Evaluating encrypted model.")
    encrypted_results = evaluate_best_model_improved(model, X_test_encrypted, y_test_subset)

    print("FINAL COMPARISON:")
    print("="*50)
    print("Standard Model Results:")
    for key, value in results.items():
        if key not in ['model', 'history']:
            print(f"  {key}: {value:.4f}")

    print("\nEncrypted Model Results:")
    for key, value in encrypted_results.items():
        print(f"  {key}: {value:.4f}")

    return best_params, best_fitness, results, encrypted_results

if __name__ == "__main__":
    main()

Dataset shape: (4240, 1)
Target distribution: [3596  644]

Original training set distribution: [2519  449]
Evaluating initial population...
Individual 1/20:
  Testing config: layers=1, neurons=[124], lr=0.006992, batch=75, epochs=111
    SSE: 150.4154, L1_penalty: 47.1531, SSE_lasso: 150.4625
  Final fitness = 150.4625
Individual 2/20:
  Testing config: layers=1, neurons=[71], lr=0.002152, batch=75, epochs=43
    SSE: 136.6082, L1_penalty: 33.7586, SSE_lasso: 136.6420
  Final fitness = 136.6420
Individual 3/20:
  Testing config: layers=1, neurons=[14], lr=0.031831, batch=75, epochs=158
    SSE: 134.3087, L1_penalty: 13.3573, SSE_lasso: 134.3221
  Final fitness = 134.3221
Individual 4/20:
  Testing config: layers=1, neurons=[124], lr=0.000897, batch=75, epochs=164
    SSE: 133.1852, L1_penalty: 47.5073, SSE_lasso: 133.2327
  Final fitness = 133.2327
Individual 5/20:
  Testing config: layers=1, neurons=[107], lr=0.000350, batch=75, epochs=75
    SSE: 128.7918, L1_penalty: 38.0510, SSE_la